In [1]:
import asyncio
import threading
import time
import concurrent.futures

In [2]:
async def fetch_data_async(task_id, delay): 
    print(f"Async task {task_id}: starting..")
    await asyncio.sleep(delay) # Non-blocking io simulation
    print(f"Async task {task_id}: done")
    return f"Data {task_id}"

async def run_async(): 
    start = time.time()
    tasks = [fetch_data_async(i,1) for i in range(5)]
    results = await asyncio.gather(*tasks)
    elapsed = time.time() - start
    print(f"Async total time: {elapsed:.2f} seconds")
    return results

In [3]:
results = await run_async()

Async task 0: starting..
Async task 1: starting..
Async task 2: starting..
Async task 3: starting..
Async task 4: starting..
Async task 0: done
Async task 1: done
Async task 2: done
Async task 3: done
Async task 4: done
Async total time: 1.00 seconds
